In [1]:
from torch import nn
import torch

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
## Downloading the database
import kagglehub


# Download latest version
path = kagglehub.dataset_download("ellipticco/elliptic-data-set")

print("Path to dataset files:", path)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/anatol/.cache/kagglehub/datasets/ellipticco/elliptic-data-set/versions/1


In [3]:
classes_df = pd.read_csv(path + "/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
features_df = pd.read_csv(path + "/elliptic_bitcoin_dataset/elliptic_txs_features.csv")
edge_df = pd.read_csv(path + "/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")

print(classes_df.head(), end="\n\n")
print(features_df.head(), end="\n\n")
print(edge_df.head())

        txId    class
0  230425980  unknown
1    5530458  unknown
2  232022460  unknown
3  232438397        2
4  230460314  unknown

   230425980  1  -0.1714692896288031  -0.18466755143291433  \
0    5530458  1            -0.171484             -0.184668   
1  232022460  1            -0.172107             -0.184668   
2  232438397  1             0.163054              1.963790   
3  230460314  1             1.011523             -0.081127   
4  230459870  1             0.961040             -0.081127   

   -1.2013688016765636  -0.12196959975910057  -0.04387454791734898  \
0            -1.201369             -0.121970             -0.043875   
1            -1.201369             -0.121970             -0.043875   
2            -0.646376             12.409294             -0.063725   
3            -1.201369              1.153668              0.333276   
4            -1.201369              1.303743              0.333276   

   -0.11300200928476244  -0.06158379407303222  -0.16209679981659642  ... 

In [4]:
print(features_df.columns)
print(classes_df.shape)
print(features_df.shape)

Index(['230425980', '1', '-0.1714692896288031', '-0.18466755143291433',
       '-1.2013688016765636', '-0.12196959975910057', '-0.04387454791734898',
       '-0.11300200928476244', '-0.06158379407303222', '-0.16209679981659642',
       ...
       '-0.5621534802884299', '-0.6009988905192808', '1.4613303209554889',
       '1.4613689382001922', '0.01827940003744589', '-0.0874901561101501',
       '-0.13115530389558736', '-0.09752359377152515', '-0.12061340670311574',
       '-0.11979245961251665'],
      dtype='object', length=167)
(203769, 2)
(203768, 167)


In [9]:
X = features_df.iloc[:, 0:95]
y = classes_df[0:-1]["class"]
print(y.shape)
print(X.shape)
print(features_df.shape)

(203768,)
(203768, 95)
(203768, 167)


In [6]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
print("Using device:", device)

Using device: mps


In [7]:
EPOCHS = 30
BATCH_SIZE = 64
LEARNING_RATE = 0.001

model = RandomForestClassifier(
                                n_estimators=100,
                                max_depth=50,
                               )

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.7655444864307798
